# Langchain with Moderation

A moderated workflow chains two models together, with the second model moderating the first.

We'll use the PromptTemplate library to create parameterized prompts.

In [10]:
import os
import torch
import numpy as np

from langchain import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

# The prompt template takes two variables: sentiment and the user message. The 
# sentiment will take values like "nice" and "very rude".

assistant_template = """
You are a {sentiment} assistant that responds to user comments,
using vocabulary similar to the user.
User:" {customer_request}"
Comment:
"""

assistant_prompt_template = PromptTemplate(
    input_variables=["sentiment", "customer_request"],
    template=assistant_template
)

# Define an intial chain - no moderator yet. The prompt template pipes into 
# the OpenAI LLM model and the model's response pipes into the output parser.

assistant_llm = ChatOpenAI(model="gpt-3.5-turbo")

output_parser = StrOutputParser()

assistant_chain = assistant_prompt_template | assistant_llm | output_parser

# Utility function to invoke the chain.
def create_dialog(customer_request, sentiment):
    assistant_response = assistant_chain.invoke(
        {"customer_request": customer_request,
        "sentiment": sentiment}
    )
    return assistant_response

We have a non-moderated chain. Let's try it out. Here's a nice reply, then a rude reply.

In [11]:
customer_request = """Learning LLM with python is a total crap experience. I feel like an Idiot!"""

response_data=create_dialog(customer_request, "nice")
print(f"nice response: {response_data}")

response_data=create_dialog(customer_request, "rude")
print(f"rude response: {response_data}")


nice response: I understand that learning LLM with python can be frustrating at times and make you feel discouraged. It's completely normal to feel that way when facing challenges. However, I assure you that with persistence and dedication, you can overcome this hurdle and become more confident in your abilities. Don't be too hard on yourself – everyone learns at their own pace. Keep pushing forward, and you'll soon see progress.
rude response: Well, it seems like you're having a bit of a rough time with learning LLM with Python. It's understandable that you may feel frustrated and even a bit foolish. But hey, don't beat yourself up too much. Learning new things can be challenging for anyone. Maybe take a step back, take a breather, and approach it with a fresh perspective. You might find that it's not as crappy as it initially seems.


The rude reply would not be good for a customer-facing bot. Let's create a moderator.

In [12]:
moderator_template = """
You are the moderator of an online forum, you are strict and will not tolerate
negative comments. If you receive an impolite comment, transform it into
something polite while mantaining the meaning if possible. If you receive a 
polite comment, repeat it word for word.

Comment: {comment_to_moderate}
"""
# We use the PromptTemplate class to create an instance of our template that will use the prompt from above and store variables we will need to input when we make the prompt.
moderator_prompt_template = PromptTemplate(
    input_variables=["comment_to_moderate"],
    template=moderator_template,
)

# Define a moderator chain. 

moderator_llm = ChatOpenAI(model="gpt-3.5-turbo")

moderator_chain = moderator_prompt_template | moderator_llm | output_parser

moderator_data = moderator_chain.invoke({"comment_to_moderate": response_data})
print(moderator_data)

Comment: It appears that you are facing some difficulties in learning LLM with Python. It is understandable that you may feel frustrated and even a bit foolish. However, please refrain from being overly critical of yourself. Learning new things can be challenging for anyone. Perhaps consider taking a moment to relax and approach it with a renewed mindset. You may discover that it is not as unpleasant as it initially appears.


Perfect. The last thing to do is to combine them into one chain. SequentialChain links chains.

In [13]:
assistant_moderated_chain = (
    {"comment_to_moderate":assistant_chain}
    |moderator_chain
)

from langchain.callbacks.tracers import ConsoleCallbackHandler

assistant_moderated_chain.invoke(
    {"sentiment": "nice", "customer_request": customer_request},
    config={'callbacks':[ConsoleCallbackHandler()]}
)

[chain/start] [1:chain:RunnableSequence] Entering Chain run with input:
{
  "sentiment": "nice",
  "customer_request": "Learning LLM with python is a total crap experience. I feel like an Idiot!"
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableParallel<comment_to_moderate>] Entering Chain run with input:
{
  "sentiment": "nice",
  "customer_request": "Learning LLM with python is a total crap experience. I feel like an Idiot!"
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableParallel<comment_to_moderate> > 3:chain:RunnableSequence] Entering Chain run with input:
{
  "sentiment": "nice",
  "customer_request": "Learning LLM with python is a total crap experience. I feel like an Idiot!"
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableParallel<comment_to_moderate> > 3:chain:RunnableSequence > 4:prompt:PromptTemplate] Entering Prompt run with input:
{
  "sentiment": "nice",
  "customer_request": "Learning LLM with python is a total crap experience. I feel l

'I understand that learning LLM with Python can be challenging at times, but please be kind to yourself. It is normal to feel overwhelmed when dealing with new concepts. With patience and practice, you will begin to feel more confident and develop a deeper understanding. Keep persevering and do not hesitate to ask for assistance or utilize resources that can facilitate the learning process for you. Remember, you are not alone on this journey!'

Let's try it from scratch.

In [14]:
assistant_moderated_chain.invoke(
    {"sentiment": "impolite", "customer_request": customer_request},
    config={'callbacks':[ConsoleCallbackHandler()]}
)

[chain/start] [1:chain:RunnableSequence] Entering Chain run with input:
{
  "sentiment": "impolite",
  "customer_request": "Learning LLM with python is a total crap experience. I feel like an Idiot!"
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableParallel<comment_to_moderate>] Entering Chain run with input:
{
  "sentiment": "impolite",
  "customer_request": "Learning LLM with python is a total crap experience. I feel like an Idiot!"
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableParallel<comment_to_moderate> > 3:chain:RunnableSequence] Entering Chain run with input:
{
  "sentiment": "impolite",
  "customer_request": "Learning LLM with python is a total crap experience. I feel like an Idiot!"
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableParallel<comment_to_moderate> > 3:chain:RunnableSequence > 4:prompt:PromptTemplate] Entering Prompt run with input:
{
  "sentiment": "impolite",
  "customer_request": "Learning LLM with python is a total crap expe

"Comment: If you find the experience of learning LLM with python to be unsatisfactory, perhaps it's simply not aligned with your interests. Additionally, feeling challenged can serve as an opportunity for growth, wouldn't you agree?"